In [ ]:
#from google.colab import drive
#drive.mount("/content/drive/", force_remount=True)
import os

In [ ]:
import pandas as pd
import re
#R = pd.DataFrame(columns=['PIPELINE/MODEL','TOKEN','LABEL'])

def run_ner(TEXT_DATA):
  #os.listdir("/content/drive/")
  #os.chdir("/content/drive/My Drive/IBM/")
  #files=os.listdir(os.getcwd())
  #for i in range(len(files)):
  #print("============= ------ ",files," ------ ==============")
  #filename = str(files)
  #text=open(os.getcwd()+"/"+files,'r')
  #TEXT_DATA = re.sub(' +', ' ',text.read())
  result=[]
  print(TEXT_DATA)
  f = flairnlp(TEXT_DATA)
  st = stanza_nlp(TEXT_DATA)
  s1,s2,s3,s4,s5,s6 = sparknlp_models(TEXT_DATA)
  #s5,s6 = sparknlp_models(TEXT_DATA)
  result.append(f)
  result.append(st)
  result.append(s1)
  result.append(s2)
  result.append(s3)
  result.append(s4)
  result.append(s5)
  result.append(s6)
  return result
    #print("Final data obtained: ",R)

In [ ]:
!pip3 install flair
from flair.data import Sentence
from flair.models import SequenceTagger

def flairnlp(text):
  print("======================   FLAIR NLP   =========================")
  #print("Before:",R)
  print("step 1")
  #text=""
  sentence = Sentence(text)
  print("step 2")
  tagger = SequenceTagger.load('ner')
  print("step 3")
  tagger.predict(sentence)
  #print(sentence)
  #print(sentence.to_dict(tag_type='ner'))
  entities = (sentence.to_dict(tag_type='ner'))["entities"]
  #print(entities)
  t=[]
  l=[]
  print(len(entities))
  for i in range(len(entities)):
    t.append((entities[i])["text"])
    listToStr = ' '.join(map(str, (entities[i])["labels"]))
    listToStr = listToStr[0:3]
    l.append(listToStr)
  return update_labels(t,l,'FLAIR')

In [ ]:
!pip3 install stanza
import stanza

def stanza_nlp(text):
  print("======================   STANZA   =========================")
  stanza.download('en')
  nlp = stanza.Pipeline(lang='en', processors='tokenize,ner')
  doc = nlp(text)
  t=[]
  l=[]
  #print(*[f'entity: {ent.text}\ttype: {ent.type}' for sent in doc.sentences for ent in sent.ents], sep='\n')
  for sent in doc.sentences:
    for ent in sent.ents:
      t.append(ent.text)
      l.append(ent.type)
  #print(t)
  #print(l)
  return update_labels(t,l,'STANZA-ONTONOTES')

In [ ]:
import os

# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed spark-nlp

#! pip install spark-nlp --upgrade

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp
import pandas as pd
from pyspark.ml import Pipeline

In [ ]:
def update_labels(tokens,labels,model_name):
  R = pd.DataFrame(columns=['PIPELINE/MODEL','TOKEN','LABEL'])
  print("=====================================================")
  print("=====================================================")
  print("=====================================================")
  print("              FOR THE MODEL ",model_name,"           ")
  print("")
  print(" Number of tokens = ",len(tokens))
  print(" Number of labels = ",len(labels))
  print("=====================================================")
  print("=====================================================")
  print("=====================================================")


  for i in range(len(labels)):
    if labels[i]=='O':
      continue
    else:
      print(tokens[i]," : ",labels[i])
      X=len(R)
      #print("Current Length: ",X)
      R.loc[X,'PIPELINE/MODEL']=model_name
      R.loc[X,'TOKEN']=tokens[i]
      R.loc[X,'LABEL']=labels[i]
      #print("New Lengrh: ",len(R))
  #print("TABLE after model ",model_name,": ")
  #print(R)
  return R

def sparknlp_models(text):

  spark=sparknlp.start()
  sp=[]
  print("======================   SPARK NLP   =========================")
  '''print("pipeline 1")
  pipeline1 = PretrainedPipeline('explain_document_dl', lang='en')
  result = pipeline1.annotate(text)
  t = list(result['token'])
  ner = list(result['ner'])
  S1 = update_labels(t,ner,'SPARK-explain_document_dl')'''
  #sp.append(S1)
  #for i in range(len(t)):
  #  print(t[i]," : ",ner[i])

  '''pipeline4 = PretrainedPipeline('explain_document_ml', lang='en')
  result = pipeline4.annotate(text)
  t = list(result['token'])
  l = list(result['ner'])
  name = update_labels(t,l,'SPARK-explain_document_ml',name)'''

  '''print("pipeline 2")
  pipeline2 = PretrainedPipeline('recognize_entities_dl', lang='en')
  result = pipeline2.annotate(text)
  t = list(result['token'])
  l = list(result['ner'])
  S2 = update_labels(t,l,'SPARK-recognize_entities_dl')'''
  #sp.append(S2)
  #for i in range(len(t)):
  #  print(t[i]," : ",ner[i])
  
  '''print("pipeline 3")
  pipeline3 = PretrainedPipeline('onto_recognize_entities_lg', lang='en')
  result = pipeline3.annotate(text)
  t = list(result['token'])
  l = list(result['ner'])
  S3 = update_labels(t,l,'SPARK-onto_recognize_entities_lg')'''
  #sp.append(S3)
  #for i in range(len(t)):
  #  print(t[i]," : ",ner[i])

  #----------- self-made pipeline

  print("pipeline 4")
  data = spark.createDataFrame([[text]]).toDF('text')
  data.show(truncate=False)
  document=DocumentAssembler().setInputCol('text').setOutputCol('document').setCleanupMode('shrink')
  sentence=SentenceDetector().setInputCols('document').setOutputCol('sentence')
  sentence.setExplodeSentences(True)
  tokenizer=Tokenizer().setInputCols('sentence').setOutputCol('token')
  checked=NorvigSweetingModel.pretrained().setInputCols(['token']).setOutputCol('checked')

  #WordEmbeddingsModel.pretrained()
  #embeddings=WordEmbeddingsModel.pretrained().setInputCols(['sentence','checked']).setOutputCol('embeddings')
  embeddings = BertEmbeddings.pretrained(name='bert_base_cased', lang='en') \
        .setInputCols(['document', 'token']) \
        .setOutputCol('embeddings')
  #ner=NerDLModel.pretrained().setInputCols(['sentence','checked','embeddings']).setOutputCol('ner')
  ner= NerDLModel.pretrained('ner_dl_bert', 'en') \
    .setInputCols(['document', 'token', 'embeddings']) \
    .setOutputCol('ner')
  converter=NerConverter().setInputCols(['sentence','checked','ner']).setOutputCol('chunk')
  pipeline=Pipeline().setStages([document,sentence,tokenizer,checked,embeddings,ner,converter])
  model=pipeline.fit(data)
  result = model.transform(data)
  #result.show()
  tokens=result.select('token.result').toPandas()
  #print(tokens['result'].values.tolist())
  ftokens = [item for sublist in tokens['result'] for item in sublist]
  #print(ftokens)
  labels=result.select('ner.result').toPandas()
  #print(labels['result'].values.tolist())
  flabels = [item for sublist in labels['result'] for item in sublist]
  #print(flabels)
  S4 = update_labels(ftokens,flabels,'SPARK-self-ner_dl_bert')
  #sp.append(S4)

  print("pipeline 5")
  embeddings=WordEmbeddingsModel.pretrained().setInputCols(['sentence','checked']).setOutputCol('embeddings')
  '''embeddings = BertEmbeddings.pretrained(name='bert_base_cased', lang='en') \
        .setInputCols(['document', 'token']) \
        .setOutputCol('embeddings')'''
  ner=NerDLModel.pretrained().setInputCols(['sentence','checked','embeddings']).setOutputCol('ner')
  '''ner= NerDLModel.pretrained('ner_dl_bert', 'en') \
    .setInputCols(['document', 'token', 'embeddings']) \
    .setOutputCol('ner')'''
  converter=NerConverter().setInputCols(['sentence','checked','ner']).setOutputCol('chunk')
  pipeline=Pipeline().setStages([document,sentence,tokenizer,checked,embeddings,ner,converter])
  model=pipeline.fit(data)
  result = model.transform(data)
  #result.show()
  tokens=result.select('token.result').toPandas()
  #print(tokens['result'].values.tolist())
  ftokens = [item for sublist in tokens['result'] for item in sublist]
  #print(ftokens)
  labels=result.select('ner.result').toPandas()
  #print(labels['result'].values.tolist())
  flabels = [item for sublist in labels['result'] for item in sublist]
  #print(flabels)
  S5 = update_labels(ftokens,flabels,'SPARK-self-ner_dl_GloVe')
  #sp.append(S5)

  print("pipeline 6")
    #WordEmbeddingsModel.pretrained()
  #embeddings=WordEmbeddingsModel.pretrained().setInputCols(['sentence','checked']).setOutputCol('embeddings')
  embeddings = BertEmbeddings.pretrained(name='bert_base_cased', lang='en') \
        .setInputCols(['document', 'token']) \
        .setOutputCol('embeddings')
  #ner=NerDLModel.pretrained().setInputCols(['sentence','checked','embeddings']).setOutputCol('ner')
  ner= NerDLModel.pretrained('onto_bert_base_cased', 'en') \
    .setInputCols(['document', 'token', 'embeddings']) \
    .setOutputCol('ner')
  converter=NerConverter().setInputCols(['sentence','checked','ner']).setOutputCol('chunk')
  pipeline=Pipeline().setStages([document,sentence,tokenizer,checked,embeddings,ner,converter])
  model=pipeline.fit(data)
  result = model.transform(data)
  #result.show()
  tokens=result.select('token.result').toPandas()
  #print(tokens['result'].values.tolist())
  ftokens = [item for sublist in tokens['result'] for item in sublist]
  #print(ftokens)
  labels=result.select('ner.result').toPandas()
  #print(labels['result'].values.tolist())
  flabels = [item for sublist in labels['result'] for item in sublist]
  #print(flabels)
  S6 = update_labels(ftokens,flabels,'SPARK-self-ner_dl_onto_bert_cased')
  #sp.append(S6)

  return S4,S5,S6

In [ ]:
#R = run_ner()

In [ ]:
def update_results(data):
  #print(data)
  new_data = pd.DataFrame(columns=['PIPELINE/MODEL','TOKEN','LABEL'])
  size=0
  for i in range(len(data)):
    #print("----- i = ",i)
    if (data.loc[i,'LABEL'])[0]=='B' and (data.loc[i,'LABEL'])[1]=='-':
      j=i+1
      #print("j = ",j)
      new_label = (data.loc[i,'LABEL'])[2:]
      #print(new_label)
      entity=""
      while j<len(data)-1 and ((data.loc[j,'LABEL'])[0]=='I' and (data.loc[j,'LABEL'])[1]=='-'):
        j=j+1
        #print("j = ",j)
      if j>i+1:
        k=i
        #print("initilazed k = ",k)
        while(k<=j):
          if k==j and (data.loc[k,'LABEL'])[0]=='B' and (data.loc[k,'LABEL'])[1]=='-':
            break
          entity=entity+" "+(data.loc[k,'TOKEN'])
          k=k+1
          #print("k = ",k)
      new_data.loc[size,'PIPELINE/MODEL'] = data.loc[i,'PIPELINE/MODEL']
      if entity!="":
        new_data.loc[size,'TOKEN'] = entity
      else:
        new_data.loc[size,'TOKEN'] = data.loc[i,'TOKEN']
      new_data.loc[size,'LABEL'] = new_label
      size=size+1
      #print("size = ",size)
  return new_data

In [ ]:
def prepare_data(R):
  X=[]
  for i in range(len(R)):
    print(R[i])
    if i==0 or i==1:
      X.append(R[i])
      continue
    X.append(update_results(R[i]))

  for i in range(len(X)):
    print("##############")
    print(X[i])

  new_data = pd.DataFrame(columns=['PIPELINE/MODEL','TOKEN','LABEL'])
  for i in range(len(X)):
    print("======================")
    print(X[i])
    df=X[i]
    for j in range(len(df)):
      x = len(new_data)
      new_data.loc[x,'PIPELINE/MODEL'] = df.loc[j,'PIPELINE/MODEL']
      new_data.loc[x,'TOKEN'] = df.loc[j,'TOKEN']
      new_data.loc[x,'LABEL'] = df.loc[j,'LABEL']
  return new_data

In [ ]:
import numpy as np
import pandas as pd
import os
import json
import re

#RESULT=[]
TEMP=[]

def main():
  '''os.listdir("/content/drive/")
  os.chdir("/content/drive/My Drive/IBM/EXPERIMENT_1/TEXT_FILES/")
  files=os.listdir()
  print(files)
  result_matrices=[]
  for i in range(len(files)):
    if(files[i]!="ik_ipr.txt"):
      continue
    print(files[i])
    text=open(os.getcwd()+"/"+files[i],'r')
    TEXT_DATA = re.sub(' +', ' ',text.read())
    TEXT_DATA = re.sub("\n\n+","\n",TEXT_DATA)'''
    #entities=['judgement','article','section','clause','paragraph','court','defendant','prosecutor','plaintiff','involved entity','advocate','appellant','learned counsel','gazette','proceedings','judge','offence','accusation','objection','testify','jurisdiction','penalty','compensation','evidence']
    #s=entities[0]
    #for j in range(len(entities)):
      #if j ==0:
        #continue
      #s=s+" "+entities[j]
    #TEXT_DATA = s
    #print(TEXT_DATA)
    #TEXT_DATA = TEXT_DATA[0:100]
    #print(TEXT_DATA)
    #TEXT_DATA = TEXT_DATA.strip("\n")
    #print(TEXT_DATA)
    #non_empty_lines = [line for line in TEXT_DATA if line.strip() != ""]
    #TEXT_DATA = non_empty_lines
    #print(TEXT_DATA)'''
  TEXT_DATA = 'However, on the basis of the material placed on record about the flagrant delay in conducting the investigation and prosecution of the accused in connection with the serious crime involving the death of a young married woman, this Court issued notice to the Director General of Police, Bihar as well as the Registrar General of the Patna High Court with a direction to them to place a report about the particulars of the present case, particularly with respect to the reasons behind such inordinate delay.'
  R = run_ner(TEXT_DATA)
  TEMP=R
    #new_data = prepare_data(R)
    #pd.set_option('display.max_rows', None)
    #for j in range(len(new_data)):
      #new_data.loc[j,'TOKEN'] = new_data.loc[j,'TOKEN'].strip()
  R = prepare_data(R)
  models = list(set(R['PIPELINE/MODEL']))
  tokens = list(set(R['TOKEN']))
  labels = list(set(R['LABEL']))
  result = pd.DataFrame(index=tokens,columns = models)
  for k in range(len(R)):
    x = R.loc[k,'PIPELINE/MODEL']
    y = R.loc[k,'TOKEN']
    z = R.loc[k,'LABEL']
    result.loc[y,x] = z
  result = result.fillna("")
    #print("========== FILE: ",files[i]," ===========")
  print(result)
  result.to_csv("result.csv",index=True)
    #result_matrices.append(result)
    #return result_matrices
main()

In [ ]:
result